# Import Libraries

In [1]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 22.6 MB/s 


In [249]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [250]:
import os
import shutil
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow_addons.activations import sparsemax
from tensorflow import keras
from keras import backend as K

from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score, f1_score, recall_score,roc_auc_score

from xgboost import XGBClassifier

pd.set_option('display.max_columns', 500)

In [251]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [252]:
filepath = "/content/gdrive/My Drive/NTU not very Deep Learning/diabetic_data.csv"

In [253]:
features = pd.read_csv(filepath)

In [254]:
print(features["patient_nbr"].value_counts())

88785891     40
43140906     28
1660293      23
88227540     23
23199021     23
             ..
11005362      1
98252496      1
1019673       1
13396320      1
175429310     1
Name: patient_nbr, Length: 71518, dtype: int64


In [255]:
features

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,MC,?,51,0,16,0,0,0,250.13,291,458,9,None,>8,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,MC,?,33,3,18,0,0,1,560,276,787,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,MC,?,53,0,9,1,0,0,38,590,296,13,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,MC,Surgery-General,45,2,21,0,0,1,996,285,998,9,None,None,No,No,No,No,No,No,Steady,No,No,Steady,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO


In [256]:
#features.drop_duplicates(subset=["patient_nbr"], inplace = True, keep = "last")

In [257]:
def change_labels_to_yes_no(x):
  if x.upper() == "NO":
    return 0
  else:
    return 1

In [258]:
features["readmitted"] = features["readmitted"].apply(change_labels_to_yes_no)

In [259]:
features["readmitted"].value_counts()

0    54864
1    46902
Name: readmitted, dtype: int64

# Get Data


In [260]:
# drop data
drop_cols = ["encounter_id", "patient_nbr", "gender", "weight", "payer_code", 
             "medical_specialty", "max_glu_serum", "repaglinide", "nateglinide", 
             "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", 
             "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", 
             "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton", 
             "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone",
             "metformin-rosiglitazone", "metformin-pioglitazone"]

features.drop(drop_cols, axis=1, inplace =True)

In [261]:
categorical_features = ["race", "age", "admission_type_id", "discharge_disposition_id", "admission_source_id", "diag_1", "diag_2", "diag_3", "A1Cresult", "metformin", "insulin", "change", "diabetesMed"] 
numeric_features = ["time_in_hospital", "num_lab_procedures", "num_procedures", "num_medications", "number_outpatient", "number_emergency", "number_inpatient", "number_diagnoses"]
recategorise = ["diag_1", "diag_2", "diag_3"]
label = "readmitted" 
all_features = features.columns.to_list()[:-1]

features["readmitted"].sum()

46902

In [262]:
def recategorize_diag(x): 
    if (x == '?'):
      return 0
    if not x.isnumeric() and x.find(".") == -1:
      return 9
    value = float(x)
    if ((value >= 390 and value <= 459) or value == 785):
      return 1
    if ((value >= 460 and value <= 519) or value == 786):
      return 2
    if ((value >= 520 and value <= 570) or value == 787):
      return 3
    if (value >= 250 and value <= 251):
      return 4
    if (value >= 800 and value <= 999):
      return 5
    if (value >= 710 and value <= 749):
      return 6
    if ((value <= 580 and value <= 629) or value == 788):
      return 7
    else:
      return 8

In [263]:
for col in recategorise:
  features[col] = features[col].apply(recategorize_diag)

In [264]:
X = features[all_features]
y = features[[label]]
raw_X_train, raw_X_test, raw_y_train, raw_y_test = train_test_split(X,y,test_size=0.2, stratify = y, random_state = 100)

# Preprocessing Pipeline

In [265]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)
categorical_transformer = OrdinalEncoder(handle_unknown = "use_encoded_value", unknown_value = 99 )

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [266]:
X_train = pd.DataFrame()
X_train[numeric_features + categorical_features] = preprocessor.fit_transform(raw_X_train)
X_test = pd.DataFrame()
X_test[numeric_features + categorical_features] = preprocessor.transform(raw_X_test)

In [267]:
X_train[categorical_features] = X_train[categorical_features].astype(int)

In [268]:
#oversampler = SMOTE(sampling_strategy = 0.50)
#X_train, raw_y_train = oversampler.fit_resample(X_train,raw_y_train)

In [269]:
label_encoder=OneHotEncoder(sparse=False)
label_encoder.fit(pd.DataFrame([[0],[1]], columns= [label]))
y_train = label_encoder.transform(raw_y_train)
y_test = label_encoder.transform(raw_y_test)

In [270]:
ds_train=tf.data.Dataset.from_tensor_slices(({i:X_train[i] for i in X_train.columns},y_train))
ds_test=tf.data.Dataset.from_tensor_slices(({i:X_test[i] for i in X_test.columns}, y_test))
ds_try=tf.data.Dataset.from_tensor_slices(({i:X_test[i] for i in X_test.columns}))

BATCH_SIZE = 800

ds_train = ds_train.shuffle(500)
ds_train = ds_train.batch(BATCH_SIZE)

ds_test = ds_test.shuffle(500)
ds_test = ds_test.batch(BATCH_SIZE)

In [271]:
feature_columns = []
feature_columns+=[tf.feature_column.numeric_column(i) for i in numeric_features]
feature_columns+=[tf.feature_column.bucketized_column(tf.feature_column.numeric_column(i),boundaries=[0.5 + j for j in range(len(X_train[i].unique())-1)]) for i in categorical_features]

# Model


In [280]:
def register_keras_custom_object(cls):
    tf.keras.utils.get_custom_objects()[cls.__name__] = cls
    return cls

def glu(x, n_units=None):
    if n_units is None:
        n_units = tf.shape(x)[-1] // 2
    return x[..., :n_units] * tf.nn.sigmoid(x[..., n_units:])


@register_keras_custom_object
@tf.function
def sparsemax(logits, axis):
    logits = tf.convert_to_tensor(logits, name="logits")

    # We need its original shape for shape inference.
    shape = logits.get_shape()
    rank = shape.rank
    is_last_axis = (axis == -1) or (axis == rank - 1)

    if is_last_axis:
        output = _compute_2d_sparsemax(logits)
        output.set_shape(shape)
        return output

    # If dim is not the last dimension, we have to do a transpose so that we can
    # still perform softmax on its last dimension.

    # Swap logits' dimension of dim and its last dimension.
    rank_op = tf.rank(logits)
    axis_norm = axis % rank
    logits = _swap_axis(logits, axis_norm, tf.math.subtract(rank_op, 1))

    # Do the actual softmax on its last dimension.
    output = _compute_2d_sparsemax(logits)
    output = _swap_axis(output, axis_norm, tf.math.subtract(rank_op, 1))

    # Make shape inference work since transpose may erase its static shape.
    output.set_shape(shape)
    return output


def _swap_axis(logits, dim_index, last_index, **kwargs):
    return tf.transpose(
        logits,
        tf.concat(
            [
                tf.range(dim_index),
                [last_index],
                tf.range(dim_index + 1, last_index),
                [dim_index],
            ],
            0,
        ),
        **kwargs,
    )


def _compute_2d_sparsemax(logits):
    """Performs the sparsemax operation when axis=-1."""
    shape_op = tf.shape(logits)
    obs = tf.math.reduce_prod(shape_op[:-1])
    dims = shape_op[-1]

    z = tf.reshape(logits, [obs, dims])

    z_sorted, _ = tf.nn.top_k(z, k=dims)

    z_cumsum = tf.math.cumsum(z_sorted, axis=-1)
    k = tf.range(1, tf.cast(dims, logits.dtype) + 1, dtype=logits.dtype)
    z_check = 1 + k * z_sorted > z_cumsum

    k_z = tf.math.reduce_sum(tf.cast(z_check, tf.int32), axis=-1)

    k_z_safe = tf.math.maximum(k_z, 1)
    indices = tf.stack([tf.range(0, obs), tf.reshape(k_z_safe, [-1]) - 1], axis=1)
    tau_sum = tf.gather_nd(z_cumsum, indices)
    tau_z = (tau_sum - 1) / tf.cast(k_z, logits.dtype)

    p = tf.math.maximum(tf.cast(0, logits.dtype), z - tf.expand_dims(tau_z, -1))

    p_safe = tf.where(
        tf.expand_dims(
            tf.math.logical_or(tf.math.equal(k_z, 0), tf.math.is_nan(z_cumsum[:, -1])),
            axis=-1,
        ),
        tf.fill([obs, dims], tf.cast(float("nan"), logits.dtype)),
        p,
    )

    p_safe = tf.reshape(p_safe, shape_op)
    return p_safe


@register_keras_custom_object
class GroupNormalization(tf.keras.layers.Layer):
    def __init__(
            self,
            groups: int = 2,
            axis: int = -1,
            epsilon: float = 1e-3,
            center: bool = True,
            scale: bool = True,
            beta_initializer="zeros",
            gamma_initializer="ones",
            beta_regularizer=None,
            gamma_regularizer=None,
            beta_constraint=None,
            gamma_constraint=None,
            **kwargs
    ):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.groups = groups
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = tf.keras.initializers.get(beta_initializer)
        self.gamma_initializer = tf.keras.initializers.get(gamma_initializer)
        self.beta_regularizer = tf.keras.regularizers.get(beta_regularizer)
        self.gamma_regularizer = tf.keras.regularizers.get(gamma_regularizer)
        self.beta_constraint = tf.keras.constraints.get(beta_constraint)
        self.gamma_constraint = tf.keras.constraints.get(gamma_constraint)
        self._check_axis()

    def build(self, input_shape):

        self._check_if_input_shape_is_none(input_shape)
        self._set_number_of_groups_for_instance_norm(input_shape)
        self._check_size_of_dimensions(input_shape)
        self._create_input_spec(input_shape)

        self._add_gamma_weight(input_shape)
        self._add_beta_weight(input_shape)
        self.built = True
        super().build(input_shape)

    def call(self, inputs, training=None):
        # Training=none is just for compat with batchnorm signature call
        input_shape = tf.keras.backend.int_shape(inputs)
        tensor_input_shape = tf.shape(inputs)

        reshaped_inputs, group_shape = self._reshape_into_groups(
            inputs, input_shape, tensor_input_shape
        )

        normalized_inputs = self._apply_normalization(reshaped_inputs, input_shape)

        outputs = tf.reshape(normalized_inputs, tensor_input_shape)

        return outputs

    def get_config(self):
        config = {
            "groups": self.groups,
            "axis": self.axis,
            "epsilon": self.epsilon,
            "center": self.center,
            "scale": self.scale,
            "beta_initializer": tf.keras.initializers.serialize(self.beta_initializer),
            "gamma_initializer": tf.keras.initializers.serialize(
                self.gamma_initializer
            ),
            "beta_regularizer": tf.keras.regularizers.serialize(self.beta_regularizer),
            "gamma_regularizer": tf.keras.regularizers.serialize(
                self.gamma_regularizer
            ),
            "beta_constraint": tf.keras.constraints.serialize(self.beta_constraint),
            "gamma_constraint": tf.keras.constraints.serialize(self.gamma_constraint),
        }
        base_config = super().get_config()
        return {**base_config, **config}

    def compute_output_shape(self, input_shape):
        return input_shape

    def _reshape_into_groups(self, inputs, input_shape, tensor_input_shape):

        group_shape = [tensor_input_shape[i] for i in range(len(input_shape))]
        group_shape[self.axis] = input_shape[self.axis] // self.groups
        group_shape.insert(self.axis, self.groups)
        group_shape = tf.stack(group_shape)
        reshaped_inputs = tf.reshape(inputs, group_shape)
        return reshaped_inputs, group_shape

    def _apply_normalization(self, reshaped_inputs, input_shape):

        group_shape = tf.keras.backend.int_shape(reshaped_inputs)
        group_reduction_axes = list(range(1, len(group_shape)))
        axis = -2 if self.axis == -1 else self.axis - 1
        group_reduction_axes.pop(axis)

        mean, variance = tf.nn.moments(
            reshaped_inputs, group_reduction_axes, keepdims=True
        )

        gamma, beta = self._get_reshaped_weights(input_shape)
        normalized_inputs = tf.nn.batch_normalization(
            reshaped_inputs,
            mean=mean,
            variance=variance,
            scale=gamma,
            offset=beta,
            variance_epsilon=self.epsilon,
        )
        return normalized_inputs

    def _get_reshaped_weights(self, input_shape):
        broadcast_shape = self._create_broadcast_shape(input_shape)
        gamma = None
        beta = None
        if self.scale:
            gamma = tf.reshape(self.gamma, broadcast_shape)

        if self.center:
            beta = tf.reshape(self.beta, broadcast_shape)
        return gamma, beta

    def _check_if_input_shape_is_none(self, input_shape):
        dim = input_shape[self.axis]
        if dim is None:
            raise ValueError(
                "Axis " + str(self.axis) + " of "
                                           "input tensor should have a defined dimension "
                                           "but the layer received an input with shape " + str(input_shape) + "."
            )

    def _set_number_of_groups_for_instance_norm(self, input_shape):
        dim = input_shape[self.axis]

        if self.groups == -1:
            self.groups = dim

    def _check_size_of_dimensions(self, input_shape):

        dim = input_shape[self.axis]
        if dim < self.groups:
            raise ValueError(
                "Number of groups (" + str(self.groups) + ") cannot be "
                                                          "more than the number of channels (" + str(dim) + ")."
            )

        if dim % self.groups != 0:
            raise ValueError(
                "Number of groups (" + str(self.groups) + ") must be a "
                                                          "multiple of the number of channels (" + str(dim) + ")."
            )

    def _check_axis(self):

        if self.axis == 0:
            raise ValueError(
                "You are trying to normalize your batch axis. Do you want to "
                "use tf.layer.batch_normalization instead"
            )

    def _create_input_spec(self, input_shape):

        dim = input_shape[self.axis]
        self.input_spec = tf.keras.layers.InputSpec(
            ndim=len(input_shape), axes={self.axis: dim}
        )

    def _add_gamma_weight(self, input_shape):

        dim = input_shape[self.axis]
        shape = (dim,)

        if self.scale:
            self.gamma = self.add_weight(
                shape=shape,
                name="gamma",
                initializer=self.gamma_initializer,
                regularizer=self.gamma_regularizer,
                constraint=self.gamma_constraint,
            )
        else:
            self.gamma = None

    def _add_beta_weight(self, input_shape):

        dim = input_shape[self.axis]
        shape = (dim,)

        if self.center:
            self.beta = self.add_weight(
                shape=shape,
                name="beta",
                initializer=self.beta_initializer,
                regularizer=self.beta_regularizer,
                constraint=self.beta_constraint,
            )
        else:
            self.beta = None

    def _create_broadcast_shape(self, input_shape):
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis] // self.groups
        broadcast_shape.insert(self.axis, self.groups)
        return broadcast_shape

In [281]:
class TransformBlock(tf.keras.Model):

    def __init__(self, features,
                 norm_type,
                 momentum=0.9,
                 virtual_batch_size=None,
                 groups=2,
                 block_name='',
                 **kwargs):
        super(TransformBlock, self).__init__(**kwargs)

        self.features = features
        self.norm_type = norm_type
        self.momentum = momentum
        self.groups = groups
        self.virtual_batch_size = virtual_batch_size

        self.transform = tf.keras.layers.Dense(self.features, use_bias=False, name=f'transformblock_dense_{block_name}')

        if norm_type == 'batch':
            self.bn = tf.keras.layers.BatchNormalization(axis=-1, momentum=momentum,
                                                         virtual_batch_size=virtual_batch_size,
                                                         name=f'transformblock_bn_{block_name}')

        else:
            self.bn = GroupNormalization(axis=-1, groups=self.groups, name=f'transformblock_gn_{block_name}')

    def call(self, inputs, training=None):
        x = self.transform(inputs)
        x = self.bn(x, training=training)
        return x


class TabNet(tf.keras.Model):

    def __init__(self, feature_columns,
                 feature_dim=64,
                 output_dim=64,
                 num_features=None,
                 num_decision_steps=5,
                 relaxation_factor=1.5,
                 sparsity_coefficient=1e-5,
                 norm_type='group',
                 batch_momentum=0.98,
                 virtual_batch_size=None,
                 num_groups=2,
                 epsilon=1e-5,
                 random_state = None,
                 **kwargs):
        """
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'batch' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(TabNet, self).__init__(**kwargs)
        if random_state is not None:
            tf.keras.utils.set_random_seed(random_state)

        # Input checks
        if feature_columns is not None:
            if type(feature_columns) not in (list, tuple):
                raise ValueError("`feature_columns` must be a list or a tuple.")

            if len(feature_columns) == 0:
                raise ValueError("`feature_columns` must be contain at least 1 tf.feature_column !")

            if num_features is None:
                num_features = len(feature_columns)
            else:
                num_features = int(num_features)

        else:
            if num_features is None:
                raise ValueError("If `feature_columns` is None, then `num_features` cannot be None.")

        if num_decision_steps < 1:
            raise ValueError("Num decision steps must be greater than 0.")

        if feature_dim <= output_dim:
            raise ValueError("To compute `features_for_coef`, feature_dim must be larger than output dim")

        feature_dim = int(feature_dim)
        output_dim = int(output_dim)
        num_decision_steps = int(num_decision_steps)
        relaxation_factor = float(relaxation_factor)
        sparsity_coefficient = float(sparsity_coefficient)
        batch_momentum = float(batch_momentum)
        num_groups = max(1, int(num_groups))
        epsilon = float(epsilon)

        if relaxation_factor < 0.:
            raise ValueError("`relaxation_factor` cannot be negative !")

        if sparsity_coefficient < 0.:
            raise ValueError("`sparsity_coefficient` cannot be negative !")

        if virtual_batch_size is not None:
            virtual_batch_size = int(virtual_batch_size)

        if norm_type not in ['batch', 'group']:
            raise ValueError("`norm_type` must be either `batch` or `group`")

        self.feature_columns = feature_columns
        self.num_features = num_features
        self.feature_dim = feature_dim
        self.output_dim = output_dim

        self.num_decision_steps = num_decision_steps
        self.relaxation_factor = relaxation_factor
        self.sparsity_coefficient = sparsity_coefficient
        self.norm_type = norm_type
        self.batch_momentum = batch_momentum
        self.virtual_batch_size = virtual_batch_size
        self.num_groups = num_groups
        self.epsilon = epsilon

        if num_decision_steps > 1:
            features_for_coeff = feature_dim - output_dim
            print(f"[TabNet]: {features_for_coeff} features will be used for decision steps.")

        if self.feature_columns is not None:
            self.input_features = tf.keras.layers.DenseFeatures(feature_columns, trainable=True)

            if self.norm_type == 'batch':
                self.input_bn = tf.keras.layers.BatchNormalization(axis=-1, momentum=batch_momentum, name='input_bn')
            else:
                self.input_bn = GroupNormalization(axis=-1, groups=self.num_groups, name='input_gn')

        else:
            self.input_features = None
            self.input_bn = None

        self.transform_f1 = TransformBlock(2 * self.feature_dim, self.norm_type,
                                           self.batch_momentum, self.virtual_batch_size, self.num_groups,
                                           block_name='f1')

        self.transform_f2 = TransformBlock(2 * self.feature_dim, self.norm_type,
                                           self.batch_momentum, self.virtual_batch_size, self.num_groups,
                                           block_name='f2')

        self.transform_f3_list = [
            TransformBlock(2 * self.feature_dim, self.norm_type,
                           self.batch_momentum, self.virtual_batch_size, self.num_groups, block_name=f'f3_{i}')
            for i in range(self.num_decision_steps)
        ]

        self.transform_f4_list = [
            TransformBlock(2 * self.feature_dim, self.norm_type,
                           self.batch_momentum, self.virtual_batch_size, self.num_groups, block_name=f'f4_{i}')
            for i in range(self.num_decision_steps)
        ]

        self.transform_coef_list = [
            TransformBlock(self.num_features, self.norm_type,
                           self.batch_momentum, self.virtual_batch_size, self.num_groups, block_name=f'coef_{i}')
            for i in range(self.num_decision_steps - 1)
        ]

        self._step_feature_selection_masks = None
        self._step_aggregate_feature_selection_mask = None

    def call(self, inputs, training=None):
        if self.input_features is not None:
            features = self.input_features(inputs)
            features = self.input_bn(features, training=training)

        else:
            features = inputs

        batch_size = tf.shape(features)[0]
        self._step_feature_selection_masks = []
        self._step_aggregate_feature_selection_mask = None

        output_aggregated = tf.zeros([batch_size, self.output_dim])
        masked_features = features
        mask_values = tf.zeros([batch_size, self.num_features])
        aggregated_mask_values = tf.zeros([batch_size, self.num_features])
        complementary_aggregated_mask_values = tf.ones(
            [batch_size, self.num_features])

        total_entropy = 0.0
        entropy_loss = 0.

        for ni in range(self.num_decision_steps):
            transform_f1 = self.transform_f1(masked_features, training=training)
            transform_f1 = glu(transform_f1, self.feature_dim)

            transform_f2 = self.transform_f2(transform_f1, training=training)
            transform_f2 = (glu(transform_f2, self.feature_dim) +
                            transform_f1) * tf.math.sqrt(0.5)

            transform_f3 = self.transform_f3_list[ni](transform_f2, training=training)
            transform_f3 = (glu(transform_f3, self.feature_dim) +
                            transform_f2) * tf.math.sqrt(0.5)

            transform_f4 = self.transform_f4_list[ni](transform_f3, training=training)
            transform_f4 = (glu(transform_f4, self.feature_dim) +
                            transform_f3) * tf.math.sqrt(0.5)

            if (ni > 0 or self.num_decision_steps == 1):
                decision_out = tf.nn.relu(transform_f4[:, :self.output_dim])

                output_aggregated += decision_out

                scale_agg = tf.reduce_sum(decision_out, axis=1, keepdims=True)

                if self.num_decision_steps > 1:
                    scale_agg = scale_agg / tf.cast(self.num_decision_steps - 1, tf.float32)

                aggregated_mask_values += mask_values * scale_agg

            features_for_coef = transform_f4[:, self.output_dim:]

            if ni < (self.num_decision_steps - 1):

                mask_values = self.transform_coef_list[ni](features_for_coef, training=training)
                mask_values *= complementary_aggregated_mask_values
                mask_values = sparsemax(mask_values, axis=-1)

                complementary_aggregated_mask_values *= (
                        self.relaxation_factor - mask_values)

                # Entropy is used to penalize the amount of sparsity in feature
                # selection.
                total_entropy += tf.reduce_mean(
                    tf.reduce_sum(
                        -mask_values * tf.math.log(mask_values + self.epsilon), axis=1)) / (
                                     tf.cast(self.num_decision_steps - 1, tf.float32))

                # Add entropy loss
                entropy_loss = total_entropy

                # Feature selection.
                masked_features = tf.multiply(mask_values, features)

                # Visualization of the feature selection mask at decision step ni
                tf.summary.image(
                     "Mask for step" + str(ni),
                     tf.expand_dims(tf.expand_dims(mask_values, 0), 3),
                     max_outputs=1)
                mask_at_step_i = tf.expand_dims(tf.expand_dims(mask_values, 0), 3)
                self._step_feature_selection_masks.append(mask_at_step_i)

            else:
                # This branch is needed for correct compilation by tf.autograph
                entropy_loss = 0.

        # Adds the loss automatically
        self.add_loss(self.sparsity_coefficient * entropy_loss)

        # Visualization of the aggregated feature importances
        tf.summary.image(
             "Aggregated mask",
             tf.expand_dims(tf.expand_dims(aggregated_mask_values, 0), 3),
             max_outputs=1)

        agg_mask = tf.expand_dims(tf.expand_dims(aggregated_mask_values, 0), 3)
        self._step_aggregate_feature_selection_mask = agg_mask

        return output_aggregated

    @property
    def feature_selection_masks(self):
        return self._step_feature_selection_masks

    @property
    def aggregate_feature_selection_mask(self):
        return self._step_aggregate_feature_selection_mask


class TabNetClassifier(tf.keras.Model):

    def __init__(self, feature_columns,
                 num_classes,
                 num_features=None,
                 feature_dim=64,
                 output_dim=64,
                 num_decision_steps=5,
                 relaxation_factor=1.5,
                 sparsity_coefficient=1e-5,
                 norm_type='group',
                 batch_momentum=0.98,
                 virtual_batch_size=None,
                 num_groups=1,
                 epsilon=1e-5,
                 random_state = None,
                 **kwargs):
        super(TabNetClassifier, self).__init__(**kwargs)

        self.num_classes = num_classes

        self.tabnet = TabNet(feature_columns=feature_columns,
                             num_features=num_features,
                             feature_dim=feature_dim,
                             output_dim=output_dim,
                             num_decision_steps=num_decision_steps,
                             relaxation_factor=relaxation_factor,
                             sparsity_coefficient=sparsity_coefficient,
                             norm_type=norm_type,
                             batch_momentum=batch_momentum,
                             virtual_batch_size=virtual_batch_size,
                             num_groups=num_groups,
                             epsilon=epsilon,
                             random_state=random_state,
                             **kwargs)

        self.clf = tf.keras.layers.Dense(num_classes, activation='softmax', use_bias=False, name='classifier')

    def call(self, inputs, training=None):
        self.activations = self.tabnet(inputs, training=training)
        out = self.clf(self.activations)

        return out

    def summary(self, *super_args, **super_kwargs):
        super().summary(*super_args, **super_kwargs)
        self.tabnet.summary(*super_args, **super_kwargs)

In [282]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
    
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def auc_score(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

# Training


In [313]:
TabNet_model=TabNetClassifier(feature_columns,
                              num_classes = 2,
                              num_features = 120,
                              feature_dim=256,
                              output_dim=32,
                              num_decision_steps= 3,
                              relaxation_factor=1.00,
                              sparsity_coefficient=1e-5,
                              virtual_batch_size=None, 
                              norm_type='group',
                              batch_momentum=0.98,
                              epsilon=1e-5)

[TabNet]: 224 features will be used for decision steps.


In [314]:
!rm -rf ./logs/
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.95e-3, decay_steps=32, decay_rate=0.9, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
TabNet_model.compile(optimizer, loss='categorical_crossentropy', metrics=[auc_score])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

TabNet_model.fit(ds_train, epochs=16 ,verbose=1, validation_data=ds_test, callbacks=[tensorboard_callback])

Epoch 1/16
102/102 [==============================] - 14s 72ms/step - loss: 0.6734 - auc_score: 0.5998 - val_loss: 0.6934 - val_auc_score: 0.6131
Epoch 2/16
102/102 [==============================] - 5s 48ms/step - loss: 0.6661 - auc_score: 0.6185 - val_loss: 0.6985 - val_auc_score: 0.6348
Epoch 3/16
102/102 [==============================] - 5s 50ms/step - loss: 0.6661 - auc_score: 0.6376 - val_loss: 0.7117 - val_auc_score: 0.6318
Epoch 4/16
102/102 [==============================] - 6s 56ms/step - loss: 0.6638 - auc_score: 0.6443 - val_loss: 0.6687 - val_auc_score: 0.6385
Epoch 5/16
102/102 [==============================] - 5s 48ms/step - loss: 0.6589 - auc_score: 0.6504 - val_loss: 0.6648 - val_auc_score: 0.6433
Epoch 6/16
102/102 [==============================] - 5s 52ms/step - loss: 0.6551 - auc_score: 0.6578 - val_loss: 0.6582 - val_auc_score: 0.6523
Epoch 7/16
102/102 [==============================] - 5s 48ms/step - loss: 0.6510 - auc_score: 0.6646 - val_loss: 0.6552 - val_au

In [315]:
Tabnet_pred_prob=TabNet_model.predict(ds_try.batch(BATCH_SIZE))
y_pred = np.argmax(Tabnet_pred_prob, axis=1)
#y_test = np.argmax(y_test, axis=1)
con_matrix=confusion_matrix(y_test,y_pred)
recall=recall_score(y_test,y_pred)
precision=precision_score(y_test,y_pred)
accuracy=accuracy_score(y_test,y_pred)
f1_score_=f1_score(y_test,y_pred)
auc_=roc_auc_score(y_test, y_pred)

print(f"{con_matrix}")
print(f"accuracy {accuracy}")
print(f"precison_score {precision}")
print(f"recall {recall}")
print(f"f1_score {f1_score_}")
print(f"auc {auc_}")

[[8397 2634]
 [4908 4414]]
accuracy 0.6294403773399498
precison_score 0.6262769580022701
recall 0.47350354001287276
f1_score 0.539279169211973
auc 0.6173609622827485


# Visualization

In [106]:
import datetime
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [165]:
TabNet_model.summary()

Model: "tab_net_classifier_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tab_net_17 (TabNet)         multiple                  1041616   
                                                                 
 classifier (Dense)          multiple                  64        
                                                                 
Total params: 1,041,680
Trainable params: 1,041,680
Non-trainable params: 0
_________________________________________________________________
Model: "tab_net_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_features_17 (DenseFea  multiple                 0         
 tures)                                                          
                                                                 
 input_gn (GroupNormalizatio  multiple                 240       
 n)            

In [287]:
!tensorboard dev upload --logdir ./logs
#!tensorboard --inspect --logdir ./logs

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/IjZWCyVvS7imox1NAMMahw/

[2022-10-02T18:17:51] Started scanning logdir.
[2022-10-02T18:17:55] Total uploaded: 112 scalars, 528 tensors (379.0 kB), 1 binary objects (946.9 kB)
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/program.py", line 276, in main
    return runner(self.flags) or 0
  File "/us

# Cross Val

In [310]:
k = 5
kf = KFold(n_splits=k, random_state=42, shuffle=True)

X = features[all_features]
y = features[[label]]

scores = pd.DataFrame(columns= ["accuracy","recall","precision","f1","auc"])
 
for train_index , test_index in kf.split(X):

  raw_X_train , raw_X_test = X.iloc[train_index,:],X.iloc[test_index,:]
  raw_y_train , raw_y_test = y.iloc[train_index, :] , y.iloc[test_index, :]

  numeric_transformer = Pipeline(
      steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
  )
  categorical_transformer = OrdinalEncoder(handle_unknown = "use_encoded_value", unknown_value = 99 )

  preprocessor = ColumnTransformer(
      transformers=[
          ("num", numeric_transformer, numeric_features),
          ("cat", categorical_transformer, categorical_features),
      ]
  )

  X_train = pd.DataFrame()
  X_train[numeric_features + categorical_features] = preprocessor.fit_transform(raw_X_train)
  X_test = pd.DataFrame()
  X_test[numeric_features + categorical_features] = preprocessor.transform(raw_X_test)

  label_encoder=OneHotEncoder(sparse=False)
  label_encoder.fit(pd.DataFrame([[0],[1]], columns= [label]))
  y_train = label_encoder.transform(raw_y_train)
  y_test = label_encoder.transform(raw_y_test)

  ds_train=tf.data.Dataset.from_tensor_slices(({i:X_train[i] for i in X_train.columns},y_train))
  ds_test=tf.data.Dataset.from_tensor_slices(({i:X_test[i] for i in X_test.columns}, y_test))
  ds_try=tf.data.Dataset.from_tensor_slices(({i:X_test[i] for i in X_test.columns}))

  BATCH_SIZE = 800

  ds_train = ds_train.shuffle(500)
  ds_train = ds_train.batch(BATCH_SIZE)

  ds_test = ds_test.shuffle(500)
  ds_test = ds_test.batch(BATCH_SIZE)

  TabNet_model=TabNetClassifier(feature_columns,
                              num_classes = 2,
                              num_features = 120,
                              feature_dim=256,
                              output_dim=32,
                              num_decision_steps= 3,
                              relaxation_factor=1.00,
                              sparsity_coefficient=1e-5,
                              virtual_batch_size=None, 
                              norm_type='group',
                              batch_momentum=0.98,
                              epsilon=1e-5)
  lr = tf.keras.optimizers.schedules.ExponentialDecay(0.95e-3, decay_steps=32, decay_rate=0.9, staircase=False)
  optimizer = tf.keras.optimizers.Adam(lr)
  TabNet_model.compile(optimizer, loss='categorical_crossentropy', metrics=[auc_score])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

  TabNet_model.fit(ds_train, epochs=16 ,verbose=1, validation_data=ds_test)

  Tabnet_pred_prob=TabNet_model.predict(ds_try.batch(BATCH_SIZE))
  y_pred = np.argmax(Tabnet_pred_prob, axis=1)
  y_test = np.argmax(y_test, axis=1)
  con_matrix=confusion_matrix(y_test,y_pred)
  recall=recall_score(y_test,y_pred)
  precision=precision_score(y_test,y_pred)
  accuracy=accuracy_score(y_test,y_pred)
  f1_score_=f1_score(y_test,y_pred)
  auc_=roc_auc_score(y_test, y_pred)
  
  temp_dict = {"accuracy":accuracy ,"recall":recall ,"precision":precision,"f1":f1_score_,"auc":auc_}
  scores = scores.append(temp_dict, ignore_index=True)

[TabNet]: 224 features will be used for decision steps.
Epoch 1/16
102/102 [==============================] - 11s 53ms/step - loss: 0.6747 - auc_score: 0.5931 - val_loss: 0.6863 - val_auc_score: 0.6336
Epoch 2/16
102/102 [==============================] - 4s 42ms/step - loss: 0.6562 - auc_score: 0.6464 - val_loss: 0.6814 - val_auc_score: 0.6474
Epoch 3/16
102/102 [==============================] - 5s 45ms/step - loss: 0.6463 - auc_score: 0.6658 - val_loss: 0.6844 - val_auc_score: 0.6546
Epoch 4/16
102/102 [==============================] - 5s 48ms/step - loss: 0.6461 - auc_score: 0.6713 - val_loss: 0.6792 - val_auc_score: 0.6595
Epoch 5/16
102/102 [==============================] - 4s 41ms/step - loss: 0.6476 - auc_score: 0.6792 - val_loss: 0.6945 - val_auc_score: 0.6558
Epoch 6/16
102/102 [==============================] - 4s 41ms/step - loss: 0.6508 - auc_score: 0.6800 - val_loss: 0.6722 - val_auc_score: 0.6689
Epoch 7/16
102/102 [==============================] - 4s 42ms/step - loss

In [311]:
scores.loc['mean'] = scores.mean()

In [312]:
scores

,accuracy,recall,precision,f1,auc
0,0.639972,0.504042,0.640059,0.563965,0.630354
1,0.626050,0.393619,0.656756,0.492228,0.609022
2,0.633666,0.497552,0.630769,0.556296,0.623947
3,0.614848,0.429877,0.620647,0.507940,0.601923
4,0.622611,0.402918,0.639755,0.494438,0.605593
mean,0.627429,0.445601,0.637597,0.522974,0.614168


# Demo Functions

In [342]:
demo_data = features.iloc[59283:59288,:].reset_index(drop = True)
#demo_data["name"] = [i for i in range(len(demo_data))]
demo_data["name"] = ["Anne Frank", "Kray Trompson", "Steffi Curry", "LeMickey James", "A. Disney"]
first_column = demo_data.pop('name')
demo_data.insert(0, 'name', first_column)

In [343]:
demo_data

,name,race,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,A1Cresult,metformin,insulin,change,diabetesMed,readmitted
0,Anne Frank,Caucasian,[50-60),3,1,1,4,37,1,22,0,0,3,9,7,1,4,None,No,No,No,No,0
1,Kray Trompson,AfricanAmerican,[60-70),1,1,7,4,43,0,6,0,0,1,7,4,1,6,None,No,Steady,No,Yes,0
2,Steffi Curry,Caucasian,[80-90),1,11,7,12,56,4,40,0,0,0,1,1,2,9,>7,No,Down,Ch,Yes,0
3,LeMickey James,Caucasian,[50-60),1,1,7,3,37,1,28,0,0,1,6,2,4,8,None,No,Steady,Ch,Yes,0
4,A. Disney,Caucasian,[60-70),3,6,1,6,74,6,47,0,0,0,1,1,1,8,None,No,Up,Ch,Yes,0


In [344]:
demo_data = demo_data.iloc[:,:-1]

In [345]:
def TabNet_Preprocessing_Steps(data: pd.DataFrame):
  demo_data = data.iloc[:,1:]
  numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
  )
  categorical_transformer = OrdinalEncoder(handle_unknown = "use_encoded_value", unknown_value = 99 )

  preprocessor = ColumnTransformer(
      transformers=[
          ("num", numeric_transformer, numeric_features),
          ("cat", categorical_transformer, categorical_features),
      ]
  )
  demo_set = pd.DataFrame()
  demo_set[numeric_features + categorical_features] = preprocessor.fit_transform(demo_data)
  return tf.data.Dataset.from_tensor_slices(({i:demo_set[i] for i in demo_set.columns}))

def Get_TabNet_Results(names, pred_proba):
  y_pred = np.argmax(pred_proba, axis=1)
  readmission = ["Yes" if i==1 else "No" for i in y_pred]
  results = pd.DataFrame()
  results['name'] = names
  results['readmission'] = readmission
  return results

# Demo

In [346]:
demo_data 

,name,race,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,A1Cresult,metformin,insulin,change,diabetesMed
0,Anne Frank,Caucasian,[50-60),3,1,1,4,37,1,22,0,0,3,9,7,1,4,None,No,No,No,No
1,Kray Trompson,AfricanAmerican,[60-70),1,1,7,4,43,0,6,0,0,1,7,4,1,6,None,No,Steady,No,Yes
2,Steffi Curry,Caucasian,[80-90),1,11,7,12,56,4,40,0,0,0,1,1,2,9,>7,No,Down,Ch,Yes
3,LeMickey James,Caucasian,[50-60),1,1,7,3,37,1,28,0,0,1,6,2,4,8,None,No,Steady,Ch,Yes
4,A. Disney,Caucasian,[60-70),3,6,1,6,74,6,47,0,0,0,1,1,1,8,None,No,Up,Ch,Yes


In [347]:
# Preprocess the information
demo_dataset = TabNet_Preprocessing_Steps(demo_data)
demo_size = len(demo_data)

# Get Prediction
prediction_results = Get_TabNet_Results(demo_data["name"] ,TabNet_model.predict(demo_dataset.batch(demo_size)))
prediction_results

,name,readmission
0,Anne Frank,Yes
1,Kray Trompson,No
2,Steffi Curry,No
3,LeMickey James,No
4,A. Disney,No
